In [1]:
## IMPORTS
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

In [3]:
## TOOL FUNCTIONS
def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].Close/df.iloc[index-30].Close
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['30WMA']/df.iloc[index-1]['30WMA']



In [4]:
#*Stage Checkers
def checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL):
    stageOneIndicator = 0
    falseReason = ""
    if volumePerc<0.7:
        stageOneIndicator+=1
    else:
        falseReason += "volume "
    if slope<=0.01 and slope>=-0.05:
        stageOneIndicator+=1
    else:
        falseReason += "slope "
    if P4WH<P4WL*1.15:
        stageOneIndicator+=1
    else:
        falseReason += "range "
    if price <= WMA*1.1 and price>= WMA*0.9:
        stageOneIndicator+=1
    else:
        falseReason += "price "
    if stageOneIndicator>=3:
        return True
    return "False " + falseReason
def checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage,prevClose):
    if volumePerc < 0.7:
        if prevStage != "Stage 2":
            #print(prevStage)
            return "Volume"
    if RS < 0.1 and prevStage != "Stage 2":
        return "RS"
    if RS < 0 and prevStage == "Stage 2":
        return "RS"
    if slope < 0.01 and prevStage != "Stage 2":
        return "Slope"
    if slope < 0.005 and prevStage == "Stage 2":
        return "Slope"
    if price < WMA*1.1 and prevStage != "Stage 2":
        return "Price"
    if price < WMA*0.95 and prevStage == "Stage 2":
        return "Price"
    return "Clear"
def checkStage(price,volumePerc, RS, slope, WMA,P4WH,P4WL,prevStage,prevClose):
    stage1Check = checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL)
    stage2Check = checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage,prevClose)
    if stage2Check == "Clear":
        return "Stage 2"
    if stage1Check == True:
        return "Stage 1 " + stage2Check 
    return stage2Check    

In [14]:
def returnStageDf(dfSorted,spDfSorted):
    deltaX = 10.4
    weights = np.arange(1,31)
    sumWeights = np.sum(weights)
    dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
    #dfSorted['30WMASlope'] = dfSorted['30WMA'].diff()*500/(dfSorted['Close'].rolling(window=104).max()-dfSorted['Close'].rolling(window=104).min())/deltaX
    dfSorted['30WMASlope'] = dfSorted.apply(lambda x: calculateSlope(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
    dfSorted['P4WH'] = dfSorted['Close'].rolling(window=4).max()
    dfSorted['P4WL'] = dfSorted['Close'].rolling(window=4).min()
    dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change()
    dfSorted['Percent'] = dfSorted.apply(lambda x: product(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
    spDfSorted['Percent'] = dfSorted.apply(lambda x: product(spDfSorted,spDfSorted.index.get_loc(x.name)),axis=1)
    dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
    dfSorted = dfSorted.dropna()
    dfSorted['Stage'] = ""
    # if dfSorted.iloc[-1]['Close']>100:
    #     return "too Expensive!!!"
    for index, element in dfSorted.iterrows():
        if dfSorted.index.get_loc(index) == 0:
            continue
        dfSorted.iloc[dfSorted.index.get_loc(index), dfSorted.columns.get_loc('Stage')] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Stage'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Close'])
    return dfSorted[["Close","Stage"]]

def getStage(ticker, weeksNum):
    # today = date.today()
    # startDate = today - timedelta(weeks=156)
    # today = today.strftime('%Y-%m-%d')
    # startDate = startDate.strftime('%Y-%m-%d')
    today = '2021-12-10'
    startDate = '1995-01-01'
    df = pdr.get_data_yahoo(ticker, start=startDate,end=today)
    sp = "^GSPC"
    spdf = pdr.get_data_yahoo(sp, start=startDate,end=today)
    sortLogic = {
        'Close': 'last',
        'Volume': 'sum'
    }
    dfSorted = df.resample("W-FRI").agg(sortLogic)
    spDfSorted = spdf.resample("W-FRI").agg(sortLogic)
    return returnStageDf(dfSorted,spDfSorted)

In [17]:
totalEarnings = 0
totalCost = 0
totalSPYEarings = 0
totalTime = 0
count = 0
totalPerc = 0
SPYPerc = 0
dfSPY = getStage('^GSPC',144)
F500 = ['GM', 'F', 'XOM', 'ETHUSD', 'GE', 'MPLX', 'MO', 'DD', 'DOCU', 'C', 'PG', 'PEP', 'AMCR', 'HPE', 'KR', 'MET', 'XLB', 'BA', 'TGT', 'ENNV', 'TKUMF', 'UPS', 'WINA', 'C', 'CNC', 'PYS', 'XRX', 'ETN', 'KODK', 'PCTY', 'MRO', 'COST', 'KO', 'WBREOX', 'JNJ', 'SIVB', 'AXP', 'SEAS', 'MMM', 'MRK', 'IP', 'CAT', 'VZ', 'LCID', 'NIO', 'LMT', 'BAC', 'SPMYY', 'ATVK', 'HD', 'MCK', 'COP', 'DAL', 'GT', 'BUD', 'BMY', 'JPM', 'ACI', 'SBAC', 'INTC', 'ADM', 'ROK', 'NUSI', 'CSCO', 'QREARX', 'IWM', 'WBA', 'TXN', 'WM', 'DIS', 'RTX', 'CSLT', 'MLM', 'TXT', 'ALGN', 'OXY', 'BAX', 'WBA', 'COUP', 'LEHLQ', 'ABT', 'TNA', 'TNA', 'MKFG.WT', 'ETHUSD', 'EMR', 'AOMOY', 'GIS', 'FDX', 'FBHS', 'EIX', 'MCD', 'SO', 'FL', 'PFE', 'UNP', 'WHR', 'PFG', 'CI', 'VIAC', 'CL', 'ANSS', 'KMB', 'AU', 'JNJ', 'LLY', 'OSRS', 'LNC', 'FMCKJ', 'JCI', 'DOW', 'NOC', 'WUMSY', 'CPB', 'MS', 'QCOM', 'SEE', 'DFS', 'PPG', 'XYL', 'EHC', 'ED', 'USB', 'AFL', 'LCID', 'LEVI', 'GS', 'HON', 'ETN', 'WFC', 'ICE', 'PEG', 'CAH', 'HAL', 'DDS', 'CMG', 'BTO', 'TXN', 'AEP', 'VOBIF', 'EVO', 'KEY', 'TWMTX', 'DOTUSD', 'CLR', 'ARMK', 'FB', 'TSN', 'MRZLF', 'FSRXU', 'W', 'WFC', 'RC.PRE', 'TCPIX', 'RHCO', 'RRD', 'GPC', 'GLW', 'CMI', 'R', 'PNC', 'ARW', 'MSFT', 'NSC', 'AKTX', 'LWLG', 'BKR', 'DOMR', 'DUK', 'MAS', 'CCK', 'RAD', 'EMN', 'ABC', 'MAN', 'FIBK', 'EXC', 'LRLCY', 'CNP', 'JWN', 'BRK.B', 'DLLFF', 'GESI', 'CP', 'PBI', 'NKE', 'SCL', 'UNH', 'GE', 'COF', 'COUP', 'GD', 'HUM', 'JLL', 'COF', 'UNM', 'CENX', 'CMS', 'CGNSF', 'PMCB', 'OVV', 'SFE', 'FUTU', 'RJF', 'TSM', 'DTP', 'RISR', 'PPWLM', 'STX', 'APD', 'ITW', 'THC', 'MARA', 'MPW', 'KMX', 'OC', 'HZNP', 'RIORX', 'DGII', 'JAZZ', 'SYUS', 'MET', 'LEA', 'FTMRQ', 'SHW', 'DOV', 'HRL', 'MPXOF', 'LPX', 'GWW', 'XOM', 'SPGI', 'BBY', 'NUE', 'UHS', 'ALLT', 'MODV', 'SHIP', 'FREE', 'AVY', 'BC', 'PFG', 'NURE', 'HTGC', 'AAPL', 'MGAWY', 'ELAMF', 'XM', 'KO', 'PARXF', 'PEBN', 'CAT', 'WMB', 'HAS', 'PSTH', 'WTM', 'ALGN', 'PKI', 'USO', 'DOTUSD', 'LUV', 'PH', 'TRV', 'PEI', 'BDX', 'CMA', 'F', 'BKR', 'XEL', 'RCII', 'ARVIX', 'DTRC', 'PGR', 'JNJ', 'TDY', 'USB', 'FE', 'KELYA', 'NYT', 'ACER', 'BLDP', 'HCYIX', 'SON', 'AES', 'YSACU', 'WING', 'JD', 'A', 'MCHP', 'FIJEX']
for symbol in F500[0:5]:
    try:
        df = getStage(symbol,1404)
        shares = 0
        sharesSPY = 0
        for index, element in df.iterrows():
            if shares == 0 and element.Stage == "Stage 2":
                shares = 1000/element.Close
                sharesSPY = 1000/dfSPY.loc[index].Close
                totalTime +=1
                totalCost +=1000
                count+=1
                print("buy "+index.strftime("%m/%d/%Y"))
                continue
            if shares != 0:
                if element.Stage == "Stage 2":
                    totalTime +=1
                else:
                    totalEarnings += shares*element.Close
                    totalSPYEarings += sharesSPY*dfSPY.loc[index].Close
                    totalPerc += shares*element.Close/1000
                    SPYPerc += sharesSPY*dfSPY.loc[index].Close/1000
                    print(symbol+" "+str(shares*element.Close))
                    print("SPY "+ str(sharesSPY*dfSPY.loc[index].Close))
                    print("sell "+index.strftime("%m/%d/%Y"))
                    shares = 0
        if shares !=0:
            totalEarnings += df.iloc[-1].Close
            totalSPYEarings += sharesSPY*dfSPY.iloc[-1].Close
            totalPerc += shares*element.Close/1000
            SPYPerc += sharesSPY*dfSPY.loc[index].Close/1000
    except:
        print(symbol+"Stock Not Found")
# totalPerc = totalEarnings/totalCost
# SPYPerc = totalSPYEarings/totalCost
# print(totalPerc)
# print(SPYPerc)
# print(totalTime)
# print(totalPerc/(totalTime/52))
# print(SPYPerc/(totalTime/52))
# print(totalCost)
print(totalPerc/count)
print(SPYPerc/count)
print(totalTime/count)
print((totalTime/500)/(10*52))

[*********************100%***********************]  1 of 1 completed
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
1995-01-03   459.209991   459.269989   457.200012   459.109985   459.109985   
1995-01-04   459.130005   460.720001   457.559998   460.709991   460.709991   
1995-01-05   460.730011   461.299988   459.750000   460.339996   460.339996   
1995-01-06   460.380005   462.489990   459.470001   460.679993   460.679993   
1995-01-09   460.670013   461.769989   459.739990   460.829987   460.829987   
...                 ...          ...          ...          ...          ...   
2021-12-03  4589.490234  4608.029785  4495.120117  4538.430176  4538.430176   
2021-12-06  4548.370117  4612.600098  4540.509766  4591.669922  4591.669922   
2021-12-07  4631.970215  4694.040039  4631.970215  4686.750000  4686.750000   
2021-12-08  4690.859863  4705.060059  4674.520020  4701.209961